### imports

In [1]:
import gc
import os
import pickle
import random
import time
from collections import Counter, defaultdict
from functools import partial
from pathlib import Path
from psutil import cpu_count
from torchvision.models import resnet50, vgg19_bn, vgg11_bn
import librosa
import numpy as np
import pandas as pd
from PIL import Image
from sklearn.model_selection import train_test_split
#from skmultilearn.model_selection import iterative_train_test_split

import torch
import torch.nn as nn
import torch.nn.functional as F
from fastprogress import master_bar, progress_bar
from torch.optim import Adam
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import transforms

In [2]:
torch.cuda.is_available()

True

In [3]:
FOLD_NO = [0]

### utils

In [4]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

SEED = 520
seed_everything(SEED)

In [5]:
#N_JOBS = cpu_count()
#os.environ['MKL_NUM_THREADS'] = str(N_JOBS)
#os.environ['OMP_NUM_THREADS'] = str(N_JOBS)
#DataLoader = partial(DataLoader, num_workers=N_JOBS)

In [6]:
# from official code https://colab.research.google.com/drive/1AgPdhSp7ttY18O3fEoHOQKlt_3HJDLi8#scrollTo=cRCaCIb9oguU
def _one_sample_positive_class_precisions(scores, truth):
    """Calculate precisions for each true class for a single sample.

    Args:
      scores: np.array of (num_classes,) giving the individual classifier scores.
      truth: np.array of (num_classes,) bools indicating which classes are true.

    Returns:
      pos_class_indices: np.array of indices of the true classes for this sample.
      pos_class_precisions: np.array of precisions corresponding to each of those
        classes.
    """
    num_classes = scores.shape[0]
    pos_class_indices = np.flatnonzero(truth > 0)
    # Only calculate precisions if there are some true classes.
    if not len(pos_class_indices):
        return pos_class_indices, np.zeros(0)
    # Retrieval list of classes for this sample.
    retrieved_classes = np.argsort(scores)[::-1]
    # class_rankings[top_scoring_class_index] == 0 etc.
    class_rankings = np.zeros(num_classes, dtype=np.int)
    class_rankings[retrieved_classes] = range(num_classes)
    # Which of these is a true label?
    retrieved_class_true = np.zeros(num_classes, dtype=np.bool)
    retrieved_class_true[class_rankings[pos_class_indices]] = True
    # Num hits for every truncated retrieval list.
    retrieved_cumulative_hits = np.cumsum(retrieved_class_true)
    # Precision of retrieval list truncated at each hit, in order of pos_labels.
    precision_at_hits = (
            retrieved_cumulative_hits[class_rankings[pos_class_indices]] /
            (1 + class_rankings[pos_class_indices].astype(np.float)))
    return pos_class_indices, precision_at_hits


def calculate_per_class_lwlrap(truth, scores):
    """Calculate label-weighted label-ranking average precision.

    Arguments:
      truth: np.array of (num_samples, num_classes) giving boolean ground-truth
        of presence of that class in that sample.
      scores: np.array of (num_samples, num_classes) giving the classifier-under-
        test's real-valued score for each class for each sample.

    Returns:
      per_class_lwlrap: np.array of (num_classes,) giving the lwlrap for each
        class.
      weight_per_class: np.array of (num_classes,) giving the prior of each
        class within the truth labels.  Then the overall unbalanced lwlrap is
        simply np.sum(per_class_lwlrap * weight_per_class)
    """
    assert truth.shape == scores.shape
    num_samples, num_classes = scores.shape
    # Space to store a distinct precision value for each class on each sample.
    # Only the classes that are true for each sample will be filled in.
    precisions_for_samples_by_classes = np.zeros((num_samples, num_classes))
    for sample_num in range(num_samples):
        pos_class_indices, precision_at_hits = (
            _one_sample_positive_class_precisions(scores[sample_num, :],
                                                  truth[sample_num, :]))
        precisions_for_samples_by_classes[sample_num, pos_class_indices] = (
            precision_at_hits)
    labels_per_class = np.sum(truth > 0, axis=0)
    weight_per_class = labels_per_class / float(np.sum(labels_per_class))
    # Form average of each column, i.e. all the precisions assigned to labels in
    # a particular class.
    per_class_lwlrap = (np.sum(precisions_for_samples_by_classes, axis=0) /
                        np.maximum(1, labels_per_class))
    # overall_lwlrap = simple average of all the actual per-class, per-sample precisions
    #                = np.sum(precisions_for_samples_by_classes) / np.sum(precisions_for_samples_by_classes > 0)
    #           also = weighted mean of per-class lwlraps, weighted by class label prior across samples
    #                = np.sum(per_class_lwlrap * weight_per_class)
    return per_class_lwlrap, weight_per_class

### dataset

In [7]:
dataset_dir = Path('../input/freesound-audio-tagging-2019')
preprocessed_dir = Path('../input/fat2019_prep_mels1')

In [8]:
csvs = {
    'train_curated': dataset_dir / 'train_curated.csv',
    'train_noisy': dataset_dir / 'train_noisy.csv',
    'sample_submission': dataset_dir / 'sample_submission.csv',
    #'argument': '../input/argument-data/label_argument_clear_large.csv'
}

dataset = {
    'train_curated': dataset_dir / 'train_curated',
    'train_noisy': dataset_dir / 'train_noisy',
    'test': dataset_dir / 'test',
}

mels = {
    'train_curated': preprocessed_dir / 'mels_train_curated.pkl',
    'train_noisy': preprocessed_dir / 'mels_trn_noisy_best50s.pkl',
    'test': preprocessed_dir / 'mels_test.pkl',  # NOTE: this data doesn't work at 2nd stage
}



In [9]:
train_curated = pd.read_csv(csvs['train_curated'])
train_noisy = pd.read_csv(csvs['train_noisy'])

train_df = pd.concat([train_curated, train_noisy], sort=True, ignore_index=True)
train_df.head()

,fname,labels
0,0006ae4e.wav,Bark
1,0019ef41.wav,Raindrop
2,001ec0ad.wav,Finger_snapping
3,0026c7cb.wav,Run
4,0026f116.wav,Finger_snapping


In [10]:
test_df = pd.read_csv(csvs['sample_submission'])
test_df.head()

,fname,Accelerating_and_revving_and_vroom,Accordion,Acoustic_guitar,Applause,Bark,Bass_drum,Bass_guitar,Bathtub_(filling_or_washing),Bicycle_bell,Burping_and_eructation,Bus,Buzz,Car_passing_by,Cheering,Chewing_and_mastication,Child_speech_and_kid_speaking,Chink_and_clink,Chirp_and_tweet,Church_bell,Clapping,Computer_keyboard,Crackle,Cricket,Crowd,Cupboard_open_or_close,Cutlery_and_silverware,Dishes_and_pots_and_pans,Drawer_open_or_close,Drip,Electric_guitar,Fart,Female_singing,Female_speech_and_woman_speaking,Fill_(with_liquid),Finger_snapping,Frying_(food),Gasp,Glockenspiel,Gong,...,Harmonica,Hi-hat,Hiss,Keys_jangling,Knock,Male_singing,Male_speech_and_man_speaking,Marimba_and_xylophone,Mechanical_fan,Meow,Microwave_oven,Motorcycle,Printer,Purr,Race_car_and_auto_racing,Raindrop,Run,Scissors,Screaming,Shatter,Sigh,Sink_(filling_or_washing),Skateboard,Slam,Sneeze,Squeak,Stream,Strum,Tap,Tick-tock,Toilet_flush,Traffic_noise_and_roadway_noise,Trickle_and_dribble,Walk_and_footsteps,Water_tap_and_faucet,Waves_and_surf,Whispering,Writing,Yell,Zipper_(clothing)
0,000ccb97.wav,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0012633b.wav,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,001ed5f1.wav,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,00294be0.wav,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,003fde7a.wav,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [11]:
labels = test_df.columns[1:].tolist()
#labels

In [12]:
num_classes = len(labels)
num_classes

80

In [13]:
y_train = np.zeros((len(train_df), num_classes)).astype(int)

for i, row in enumerate(train_df['labels'].str.split(',')):
    for label in row:
        idx = labels.index(label)
        y_train[i, idx] = 1

with open('../input/mix-data-argument/label_argument.pickle', 'rb') as argument:
    y_train_argument = pickle.load(argument)
    y_train_argument = np.array(y_train_argument)
        
print(y_train.shape, y_train_argument.shape)

(24785, 80) (4703, 80)


In [14]:
#with open('../input/all-data/train_curated.pickle', 'rb') as curated:
    #x_train = pickle.load(curated)

#for i in range(5):
    #with open('../input/all-data/train_noisy_{}.pickle'.format(i+1), 'rb') as noisy:
        #x_train.extend(pickle.load(noisy))

#with open('../input/mix-data-argument/argument.pickle', 'rb') as argument:
   # x_train_argument = pickle.load(argument)

#len(x_train), len(x_train_argument)    

In [15]:
with open('../input/all-data/test.pickle', 'rb') as test:
    x_test = pickle.load(test)

In [16]:
class FATTrainDataset(Dataset):
    def __init__(self, mels, labels, transforms):
        super().__init__()
        self.mels = mels
        self.labels = labels
        self.transforms = transforms
        
    def __len__(self):
        return len(self.mels)
    
    def __getitem__(self, idx):
        
        image = Image.fromarray(self.mels[idx], mode='RGB')        
        time_dim, base_dim = image.size
        if time_dim == base_dim:
            image1, image2, image3 = image, image, image
            image1 = self.transforms(image1).div_(255)
            image2 = self.transforms(image2).div_(255)
            image3 = self.transforms(image3).div_(255)
        elif time_dim > base_dim and time_dim < 3 * base_dim:
            image1 = image.crop([0, 0, base_dim, base_dim])
            crop1 = random.randint(0, time_dim - base_dim)
            image2 = image.crop([crop1, 0, crop1 + base_dim, base_dim])
            crop2 = random.randint(0, time_dim - base_dim)
            image3 = image.crop([crop2, 0, crop2 + base_dim, base_dim])
            image1 = self.transforms(image1).div_(255)
            image2 = self.transforms(image2).div_(255)
            image3 = self.transforms(image3).div_(255)
        else:
            crop1 = random.randint(0, int(time_dim/3))
            image1 = image.crop([crop1, 0, crop1 + base_dim, base_dim])
            crop2 = random.randint(int(time_dim/3), int(time_dim*(2/3)))
            image2 = image.crop([crop2, 0, crop2 + base_dim, base_dim])
            crop3 = random.randint(int(time_dim*(2/3)), time_dim - base_dim)
            image3 = image.crop([crop3, 0, crop3+ base_dim, base_dim])
            image1 = self.transforms(image1).div_(255)
            image2 = self.transforms(image2).div_(255)
            image3 = self.transforms(image3).div_(255)
            
        label = self.labels[idx]
        label = torch.from_numpy(label).float()
        
        return [image1, image2, image3], label

In [17]:
class FATTestDataset(Dataset):
    def __init__(self, fnames, mels, transforms, tta=5):
        super().__init__()
        self.fnames = fnames
        self.mels = mels
        self.transforms = transforms
        self.tta = tta
        
    def __len__(self):
        return len(self.fnames) * self.tta
    
    def __getitem__(self, idx):
        new_idx = idx % len(self.fnames)
        
        image = Image.fromarray(self.mels[new_idx], mode='RGB') 
        time_dim, base_dim = image.size
        if time_dim == base_dim:
            image1, image2, image3 = image, image, image
            image1 = self.transforms(image1).div_(255)
            image2 = self.transforms(image2).div_(255)
            image3 = self.transforms(image3).div_(255)
        elif time_dim > base_dim and time_dim < 3 * base_dim:
            image1 = image.crop([0, 0, base_dim, base_dim])
            crop1 = random.randint(0, time_dim - base_dim)
            image2 = image.crop([crop1, 0, crop1 + base_dim, base_dim])
            crop2 = random.randint(0, time_dim - base_dim)
            image3 = image.crop([crop2, 0, crop2 + base_dim, base_dim])
            image1 = self.transforms(image1).div_(255)
            image2 = self.transforms(image2).div_(255)
            image3 = self.transforms(image3).div_(255)
        else:
            crop1 = random.randint(0, int(time_dim/3))
            image1 = image.crop([crop1, 0, crop1 + base_dim, base_dim])
            crop2 = random.randint(int(time_dim/3), int(time_dim*(2/3)))
            image2 = image.crop([crop2, 0, crop2 + base_dim, base_dim])
            crop3 = random.randint(int(time_dim*(2/3)), time_dim - base_dim)
            image3 = image.crop([crop3, 0, crop3+ base_dim, base_dim])
            image1 = self.transforms(image1).div_(255)
            image2 = self.transforms(image2).div_(255)
            image3 = self.transforms(image3).div_(255)
        
        fname = self.fnames[new_idx]
        
        return [image1, image2, image3], fname

In [18]:
transforms_dict = {
    'train': transforms.Compose([
        transforms.RandomHorizontalFlip(0.5),
        transforms.ToTensor(),
    ]),
    'test': transforms.Compose([
        #transforms.RandomHorizontalFlip(0.3),
        transforms.ToTensor(),
    ]),
}

### model

In [19]:
class ConvBlock(nn.Module):

    def __init__(self, in_channels, out_channels):
        super(ConvBlock, self).__init__()
        self.conv_max = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, 3, 1, 1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(),
            nn.Conv2d(out_channels, out_channels, 3, 1, 1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(),
            nn.MaxPool2d(2, stride=2))

        self._init_weights()

    def _init_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight)
                if m.bias is not None:
                    nn.init.zeros_(m.bias)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.zeros_(m.bias)

    def forward(self, x):

        return self.conv_max(x)


class CLDNN(nn.Module):

    def __init__(self):
        super(CLDNN, self).__init__()
        self.preprocess = nn.Sequential(nn.BatchNorm2d(3), nn.ReLU())
        self.cnn = nn.Sequential(ConvBlock(3, 32),
                                 ConvBlock(32, 64),
                                 ConvBlock(64, 128),
                                 ConvBlock(128, 256),
                                 ConvBlock(256, 512),
                                 nn.AvgPool2d((4,4))) 

    def forward(self, x):
        
        x = self.preprocess(x)
        x = self.cnn(x)
        x = x.view(x.size(0), -1)

        return x

In [20]:
class main(nn.Module):
    
    def __init__(self):
        super(main, self).__init__()
        self.fea_extractor = CLDNN()
        self.cls = nn.Sequential(nn.Dropout(0.5),
                                 nn.Linear(512, 80),
                                 nn.Sigmoid())
        self.att= nn.Sequential(nn.Dropout(0.5),
                                nn.Linear(512, 80),
                                nn.Sigmoid())
    
    def forward(self, image1, image2, image3):
        
        fea1 = self.fea_extractor(image1)
        fea2 = self.fea_extractor(image2)
        fea3 = self.fea_extractor(image3)
        
        fea1 = torch.unsqueeze(fea1, dim=1)
        fea2 = torch.unsqueeze(fea2, dim=1)
        fea3 = torch.unsqueeze(fea3, dim=1)
        
        x = torch.cat((fea1, fea2, fea3), dim=1)
        
        x_reshape = x.contiguous().view(-1, x.size(-1))
        x_cls = self.cls(x_reshape)
        x_att = self.att(x_reshape)
        x_cls = x_cls.view(-1, x.size(1), x_cls.size(-1))
        x_att = x_att.view(-1, x.size(1), x_att.size(-1))
        att = x_att / torch.sum(x_att, dim=1, keepdim=True)
        output = att * x_cls
        output = torch.sum(output, dim=1, keepdim=False)
        output = torch.clamp(output, 1e-8, 1)
        
        return output

### train

In [21]:
def train_model(x_train, y_train, train_index, val_index, n_fold, train_transforms):
    
    num_epochs = 100
    batch_size = 32
    test_batch_size = 32
    lr = 1e-3
    eta_min = 1e-5
    t_max = 5
    
    num_classes = y_train.shape[1]

    x_trn= [x_train[idx] for idx in train_index] + x_train_argument
    x_val= [x_train[idx] for idx in val_index]
    y_trn= np.asarray([y_train[idx] for idx in train_index])
    y_trn= np.vstack((y_trn, y_train_argument))
    y_val= [y_train[idx] for idx in val_index]
    
    train_dataset = FATTrainDataset(x_trn, y_trn, train_transforms)
    valid_dataset = FATTrainDataset(x_val, y_val, train_transforms)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    valid_loader = DataLoader(valid_dataset, batch_size=test_batch_size, shuffle=False)
    
    model = main().cuda()
    weight = [0.4]*num_classes
    weight = torch.FloatTensor(weight)
    criterion = nn.BCELoss(weight).cuda()
    optimizer = Adam(params=model.parameters(), lr=lr, amsgrad=False, weight_decay=1e-6)
    scheduler = CosineAnnealingLR(optimizer, T_max=t_max, eta_min=eta_min)

    best_epoch = -1
    best_lwlrap = 0.
    #mb = master_bar(range(num_epochs))
    print('Start Training')
    for epoch in range(num_epochs):
        start_time = time.time()
        model.train()
        avg_loss = 0.
        
        for inputs, labels in train_loader:
            image1, image2, image3 = inputs
            image1, image2, image3 = image1.cuda(), image2.cuda(), image3.cuda()
            preds = model(image1, image2, image3)
            loss = criterion(preds, labels.cuda())

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            avg_loss += loss.item() / len(train_loader)

        model.eval()
        valid_preds = np.zeros((len(x_val), num_classes))
        avg_val_loss = 0.

        for i, (inputs, labels) in enumerate(valid_loader):
            image1, image2, image3 = inputs
            image1, image2, image3 = image1.cuda(), image2.cuda(), image3.cuda()
            preds = model(image1, image2, image3)
            loss = criterion(preds, labels.cuda())
            valid_preds[i * test_batch_size: (i+1) * test_batch_size] = preds.cpu().detach().numpy()

            avg_val_loss += loss.item() / len(valid_loader)
            
        score, weight = calculate_per_class_lwlrap(np.asarray(y_val), valid_preds)
        lwlrap = (score * weight).sum()
        
        scheduler.step()

        if (epoch + 1) % 1 == 0:
            elapsed = time.time() - start_time
            print(f'Epoch {epoch+1} - avg_train_loss: {avg_loss:.4f}  avg_val_loss: {avg_val_loss:.4f}  val_lwlrap: {lwlrap:.6f}  time: {elapsed:.0f}s')
    
        if lwlrap > best_lwlrap:
            best_epoch = epoch + 1
            best_lwlrap = lwlrap
            torch.save(model.state_dict(), 'weight_best_{}.pt'.format(n_fold))
            
    return {
        'best_epoch': best_epoch,
        'best_lwlrap': best_lwlrap,
    }

In [22]:
#train_s = pd.read_csv('../input/train-stratified/train_stratified.csv')

In [23]:
#for FOLD in FOLD_NO:
    #val_index = train_s[train_s.fold == FOLD].index
    #tra_index = train_s[train_s.fold != FOLD].index
    #result = train_model(x_train, y_train, tra_index, val_index, FOLD, transforms_dict['train'])
   # print(result)

### predict

In [24]:
def predict_model(test_fnames, x_test, test_transforms, n_fold, num_classes, tta=5):
    #torch.cuda.empty_cache() 
    batch_size = 32

    test_dataset = FATTestDataset(test_fnames, x_test, test_transforms, tta=tta)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    model = main().cuda()
    model.load_state_dict(torch.load('../input/model-3-2/weight_best_{}.pt'.format(n_fold)))
    model.cuda()
    model.eval()

    all_outputs, all_fnames = [], []

    pb = progress_bar(test_loader)
    for inputs, fnames in pb:
        image1, image2, image3 = inputs
        image1, image2, image3 = image1.cuda(), image2.cuda(), image3.cuda()
        preds = model(image1, image2, image3).detach()
        all_outputs.append(preds.cpu().numpy())
        all_fnames.extend(fnames)

    test_preds = pd.DataFrame(data=np.concatenate(all_outputs),
                              index=all_fnames,
                              columns=map(str, range(num_classes)))
    test_preds = test_preds.groupby(level=0).mean()

    return test_preds

In [25]:
#torch.cuda.empty_cache() 

In [26]:
test_preds = np.zeros((len(x_test),80))
for fold in [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]:
    test_preds += predict_model(test_df['fname'], x_test, transforms_dict['test'], fold, num_classes, tta=25).values
test_preds /= 10

In [27]:
test_df[labels] = test_preds
test_df.to_csv('submission.csv', index=False)
test_df.head()

,fname,Accelerating_and_revving_and_vroom,Accordion,Acoustic_guitar,Applause,Bark,Bass_drum,Bass_guitar,Bathtub_(filling_or_washing),Bicycle_bell,Burping_and_eructation,Bus,Buzz,Car_passing_by,Cheering,Chewing_and_mastication,Child_speech_and_kid_speaking,Chink_and_clink,Chirp_and_tweet,Church_bell,Clapping,Computer_keyboard,Crackle,Cricket,Crowd,Cupboard_open_or_close,Cutlery_and_silverware,Dishes_and_pots_and_pans,Drawer_open_or_close,Drip,Electric_guitar,Fart,Female_singing,Female_speech_and_woman_speaking,Fill_(with_liquid),Finger_snapping,Frying_(food),Gasp,Glockenspiel,Gong,...,Harmonica,Hi-hat,Hiss,Keys_jangling,Knock,Male_singing,Male_speech_and_man_speaking,Marimba_and_xylophone,Mechanical_fan,Meow,Microwave_oven,Motorcycle,Printer,Purr,Race_car_and_auto_racing,Raindrop,Run,Scissors,Screaming,Shatter,Sigh,Sink_(filling_or_washing),Skateboard,Slam,Sneeze,Squeak,Stream,Strum,Tap,Tick-tock,Toilet_flush,Traffic_noise_and_roadway_noise,Trickle_and_dribble,Walk_and_footsteps,Water_tap_and_faucet,Waves_and_surf,Whispering,Writing,Yell,Zipper_(clothing)
0,000ccb97.wav,0.000003,0.000003,0.000004,3.881384e-05,0.000245,0.000554,0.000015,0.000383,0.000411,0.000080,2.552399e-04,0.000127,0.000050,7.710323e-06,0.000922,0.000044,0.000500,0.001932,0.000091,0.000402,0.000141,0.000557,0.000296,3.234178e-06,0.000261,0.001286,0.000025,0.000042,3.929060e-03,1.157989e-06,0.000049,0.000109,0.001635,0.000062,0.003898,0.000033,0.000100,0.000030,0.000074,...,2.372459e-05,0.004639,0.003743,0.032774,0.000209,7.210171e-05,6.670536e-05,0.000193,0.000047,0.000103,2.643386e-04,0.000100,0.000040,0.000085,1.161818e-04,6.903465e-03,0.000062,0.069070,9.370479e-06,0.001314,0.000197,0.000023,0.000054,0.000078,0.000042,0.000055,0.000087,1.455830e-05,0.000599,0.000087,0.000029,5.335552e-06,0.000146,0.000071,7.382637e-05,0.000014,0.004418,0.000231,9.741044e-05,0.000394
1,0012633b.wav,0.147627,0.000147,0.000865,3.355853e-04,0.002570,0.006165,0.013212,0.002365,0.001470,0.002294,3.185084e-03,0.044947,0.016188,3.417143e-04,0.007329,0.005974,0.001518,0.000647,0.000801,0.000906,0.000967,0.003929,0.000239,1.999887e-03,0.006685,0.000576,0.000147,0.004130,1.785490e-04,3.987861e-03,0.098317,0.001815,0.000947,0.000466,0.001038,0.002703,0.037066,0.000350,0.004353,...,3.685788e-03,0.017879,0.015805,0.007627,0.001855,5.472174e-03,2.632296e-03,0.001463,0.003733,0.001298,1.527046e-03,0.277326,0.001411,0.007838,4.303717e-03,6.128678e-04,0.026317,0.001683,3.234287e-03,0.013181,0.050792,0.000346,0.000872,0.001160,0.003319,0.002837,0.000292,8.578656e-04,0.001018,0.000187,0.002656,7.327512e-03,0.000220,0.002026,2.817482e-04,0.003890,0.002920,0.002624,2.004767e-03,0.265863
2,001ed5f1.wav,0.001014,0.000149,0.000107,1.933985e-04,0.000363,0.003299,0.000148,0.005646,0.000458,0.000754,5.528304e-04,0.000763,0.000778,2.692185e-04,0.000556,0.000101,0.002077,0.000605,0.000816,0.021549,0.001733,0.000983,0.000096,7.436065e-05,0.011859,0.000266,0.000748,0.010906,1.117396e-03,5.009715e-04,0.000372,0.000085,0.000094,0.001203,0.027560,0.000702,0.000561,0.000450,0.000321,...,3.689144e-04,0.000497,0.008851,0.000369,0.074831,2.219649e-04,5.702501e-05,0.006644,0.000907,0.000512,1.314641e-02,0.000232,0.000975,0.000162,1.069678e-04,2.622573e-03,0.502255,0.002428,2.413007e-04,0.000300,0.000518,0.000135,0.000646,0.101509,0.000635,0.004207,0.000193,8.591702e-05,0.074790,0.000846,0.016068,2.797624e-04,0.000182,0.000796,2.090034e-04,0.000536,0.000210,0.000353,3.424776e-05,0.000483
3,00294be0.wav,0.000009,0.000003,0.000039,1.519812e-08,0.000072,0.000068,0.000002,0.000014,0.000047,0.000080,7.094192e-07,0.000331,0.000001,1.044664e-08,0.000901,0.000025,0.000185,0.000001,0.000010,0.000193,0.000004,0.000120,0.001738,1.774922e-07,0.000002,0.000169,0.000005,0.000215,3.451510e-07,2.915498e-08,0.000121,0.000022,0.000137,0.000033,0.000032,0.000019,0.000009,0.000005,0.000012,...,5.880924e-07,0.000001,0.000018,0.000001,0.000008,6.827956e-07,3.514860e-07,0.000048,0.000065,0.011533,3.967460e-08,0.000168,0.000069,0.999993

In [28]:
#test_df.head(20)